In [1]:
##IMPORT PACKAGES
%matplotlib inline

from pathlib import Path

import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
import matplotlib.pylab as plt

In [2]:
##IMPORT AND TRANSFORM DATA

#import .csv and create cust_test_df
ubank_df = pd.read_csv('UniversalBank.csv')
data = {'ID': [5001], 'ZIP.Code': [43016],
                          'Age': [40],
                          'Experience': [10],
                          'Income': [84],
                          'Family': [2],
                          'CCAvg': [2],
                          'Mortgage': [0],
                          'Securities Account': [0],
                          'CD Account': [0],
                          'Online': [1],
                          'CreditCard': [1],
                          'Education_1': [0],
                          'Education_2': [1],
                          'Education_3': [0],
                          'Personal Loan': [1]}
cust_test_df = pd.DataFrame(data=data)

#converting Education into dummy variable and adding to dataset
dummy = pd.get_dummies(ubank_df['Education']).rename(columns=lambda x: 'Education_' + str(x))
ubank_df = pd.concat([ubank_df, dummy], axis =1)
ubank_df = ubank_df.drop(['Education'],axis=1)

#reorder columns so PersonalLoan is [:-1]
ubank_df = ubank_df[['ID', 'ZIP Code', 'Age', 'Experience', 'Income', 'Family', 'CCAvg',
       'Mortgage', 'Securities Account', 'CD Account', 'Online', 'CreditCard', 'Education_1', 'Education_2', 'Education_3', 'Personal Loan']]

In [9]:
##SPLIT AND SCALE
#split
X = ubank_df.iloc[:,2:-1]
y = ubank_df.iloc[:,-1]
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.4, random_state=6011)

X_cust = cust_test_df.iloc[:,2:-1]
y_cust = pd.DataFrame(data=cust_test_df.iloc[:,-1])

#scaling
scaled_X_train = X_train
col_names = ['Age','Experience','Income','Family','CCAvg','Mortgage']
scaled_X_train = X_train[col_names]
scaler = StandardScaler().fit(scaled_X_train.values)
scaled_X_train = scaler.transform(scaled_X_train.values)
X_train.loc[:,:6] = scaled_X_train

C:\Users\harri\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [8]:
##CLASSIFICATION FOR CUSTOMER
classifier = KNeighborsClassifier(n_neighbors=1)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_cust)
print(y_pred)
#This customer would be classified as likely to accept a personal loan if offered.

[1]


In [ ]:
##CLASSIFICATION AND CONFUSION MATRIX FOR BEST K
classifier = KNeighborsClassifier(n_neighbors=1)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_cust)

#This customer would be classified as likely to accept a personal loan if offered.

y_pred = classifier.predict(X_valid)
print(confusion_matrix(y_valid, y_pred))
print(classification_report(y_valid, y_pred))